# Downloading festival image dataset from github

In [26]:
import os
import zipfile
import shutil

url = "https://github.com/Ishaanshri95/datasets/archive/refs/heads/main.zip"


!wget --no-check-certificate \
    "https://github.com/Ishaanshri95/datasets/archive/refs/heads/main.zip"


dir = 'tmp'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

zip_ref = zipfile.ZipFile('main.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('tmp') #Extracts the files into the tmp folder
zip_ref.close()
os.remove('main.zip')
shutil.move('/content/tmp/datasets-main/festivals', '/content/festivals')
shutil.rmtree('tmp')
shutil.rmtree('festivals/festivals')

--2022-02-13 13:51:18--  https://github.com/Ishaanshri95/datasets/archive/refs/heads/main.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Ishaanshri95/datasets/zip/refs/heads/main [following]
--2022-02-13 13:51:18--  https://codeload.github.com/Ishaanshri95/datasets/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 13.112.159.149
Connecting to codeload.github.com (codeload.github.com)|13.112.159.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [            <=>     ]  51.97M  14.2MB/s    in 4.5s    

2022-02-13 13:51:23 (11.6 MB/s) - ‘main.zip’ saved [54492797]



# Some imports, setting the device

In [27]:
import os
import zipfile
import shutil
import numpy as np

import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms as transforms

from tqdm import tqdm

import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

device

'cuda'

In [28]:

# festival_list_class = [i for i in range(len(festival_list))]
# converter = dict(zip(festival_list, festival_list_class))
# festival_data = []
# festival_target = []

# for fest in festival_list:
#   print()
#   print(fest)
#   imgs = next(os.walk(root+'/'+fest))[2]

#   for img in imgs:
#     img_data = cv2.imread(root+'/'+fest+'/'+img)
#     img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
#     img_data = cv2.resize(img_data, (100,100), interpolation=cv2.INTER_LINEAR)
#     # print(img_data.shape)

#     festival_data.append(img_data)
#     targ = [0 for i in festival_list]
#     targ[converter[fest]] = 1
#     festival_target.append(targ)

# festival_data = np.array(festival_data)
# festival_target = np.array(festival_target)
# festival_data = torch.tensor(festival_data).to(device)
# festival_target = torch.tensor(festival_target).to(device)
# # festival_data = np.array(festival_data)                    # throws deprecation warning

# Loading the dataset, making batch loaders

In [29]:
import torchvision

root = '/content/festivals'

def get_data_loader(root=root, batch_size = 32, train=True):
  transform = {
    'train': transforms.Compose([
      transforms.Resize([100,100]), # Resizing the image as the VGG only take 224 x 244 as input size
      #TODO if it is needed, add the random crop
      transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
      transforms.Resize([100,100]),
      transforms.ToTensor()
    ])
  }
  tt = lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)
  # data = torchvision.datasets.ImageFolder(root=root, transform=transform['train'] if train else 'test', target_transform=tt)
  data = torchvision.datasets.ImageFolder(root=root, transform=transform['train'] if train else transform['test'])
  data_loader = DataLoader(dataset=data, batch_size=batch_size, shuffle=True, num_workers=2)
  
  # print(data)
  return data_loader


torch.multiprocessing.freeze_support()
train_loader = get_data_loader()
test_loader = get_data_loader(train=False)

# print(type(test_loader))
for i in range(10):
  batch_x, batch_y = next(iter(test_loader))
  print(np.shape(batch_x), batch_y.shape) 

root = 'festivals'
# # shutil.rmtree('tmp')
festival_list = next(os.walk(root))[1]


torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])
torch.Size([32, 3, 100, 100]) torch.Size([32])


# Defining Hyperparameters

In [30]:
num_classes = len(festival_list)
batch_size = 16                        # besties don't let each other use batches larger than 32 :)
learning_rate = 0.001
num_epochs = 40

# Defining CNN class
Note: model takes in input layer of size 100x100.

In [31]:
class CNN(nn.Module):
  def __init__(self, num_classes):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    self.conv2 = nn.Conv2d(in_channels = 24, out_channels = 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    self.conv3 = nn.Conv2d(in_channels = 48, out_channels = 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    self.conv4 = nn.Conv2d(in_channels = 96, out_channels = 192, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    self.fc1 = nn.Linear(192*5*5, 400)
    self.fc2 = nn.Linear(400, num_classes)
  
  def forward(self, x):
    x = self.pool(self.conv1(x))           # convolutional layer backbone
    x = self.pool(self.conv2(x))           # convolutional layer backbone
    x = self.pool(self.conv3(x))           # convolutional layer backbone
    x = self.pool(self.conv4(x))           # convolutional layer backbone
    x = x.reshape(x.shape[0], -1)        # Fully connected neural network layers
    x = F.relu(self.fc1(x))              # Fully connected neural network layers
    x = self.fc2(x)                      # Fully connected neural network layers

    return x

# Creating the CNN model

In [32]:
x = torch.rand(64, 3, 100, 100)                    # 100 x 100 images with 3 RGB channels
x = x.to(device)                                   # loads data onto gpu if available
model = CNN(num_classes=num_classes).to(device)    # creates model
model(x).shape                                     # checks if layer sizes are consistent with input and target

torch.Size([64, 7])

# Loss and Optimizer

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:                    # takes one batch from the data loader
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)     # finds which index of the predicted target is maximum    [num_samples]
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

for epoch in range(num_epochs):
  # print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    data = data.to(device)
    targets = targets.to(device)

    data = data.reshape(data.shape[0], 3, 100, 100)

    scores = model(data)
    loss = criterion(scores, targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()


  1%|          | 1/133 [00:00<00:30,  4.40it/s]

# Evaluate model accuracy on train and test

In [14]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            # x = x.reshape(x.shape[0], -1)    # dont include this line in cnn

            scores = model(x)
            _, predictions = scores.max(1)
            # print(num_correct.shape)

            # print(predictions.shape)
            # print(scores.shape)
            # print(y.shape)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples

print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.89
Accuracy on test set: 98.89
